- target: wide field, beam 14 should include GRB221009a, verify with catalogues
- observation log: multiple, see ASKAP folder

# 0. Defining Variables

In [ ]:
# extract measurement set
homeDir = 'Volumes/G-RAID_Thunderbold_4/xiao_data/ASKAP'

tar -xvf scienceData.AS113_67.SB44857.AS113_67.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.AS113_68.SB44918.AS113_68.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.AS113_69.SB45060.AS113_69.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.AS113_70.SB45086.AS113_70.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.AS113_71.SB45416.AS113_71.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.CRACOTest_A.SB46350.CRACOTest_A.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.CRACOTest_A.SB46419.CRACOTest_A.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.CRACOTest_B.SB46492.CRACOTest_B.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.CRACOTest_B.SB46554.CRACOTest_B.beam14_averaged_cal.leakage.ms.tar
tar -xvf scienceData.CRACO_ULP2.SB48611.CRACO_ULP2.beam14_averaged_cal.leakage.ms.tar

In [ ]:
# make subdirectories for each observation
mkdir 44857 &&
mkdir 44918 &&
mkdir 45060 &&
mkdir 45086 &&
mkdir 45416 &&
mkdir 46350 &&
mkdir 46419 &&
mkdir 46492 &&
mkdir 46554 &&
mkdir 48511

In [ ]:
# move files into subdirectories
mv scienceData.AS113_67.SB44857.AS113_67.beam14_averaged_cal.leakage.ms 44857 &&
mv scienceData.AS113_67.SB44857.AS113_67.beam14_averaged_cal.leakage.ms.tar 44857 &&

mv scienceData.AS113_68.SB44918.AS113_68.beam14_averaged_cal.leakage.ms 44918 &&
mv scienceData.AS113_68.SB44918.AS113_68.beam14_averaged_cal.leakage.ms.tar 44918 &&

mv scienceData.AS113_69.SB45060.AS113_69.beam14_averaged_cal.leakage.ms 45060 &&
mv scienceData.AS113_69.SB45060.AS113_69.beam14_averaged_cal.leakage.ms.tar 45060 &&

mv scienceData.AS113_70.SB45086.AS113_70.beam14_averaged_cal.leakage.ms 45086 &&
mv scienceData.AS113_70.SB45086.AS113_70.beam14_averaged_cal.leakage.ms.tar 45086 &&

mv scienceData.AS113_71.SB45416.AS113_71.beam14_averaged_cal.leakage.ms 45416 &&
mv scienceData.AS113_71.SB45416.AS113_71.beam14_averaged_cal.leakage.ms.tar 45416 &&

mv scienceData.CRACOTest_A.SB46350.CRACOTest_A.beam14_averaged_cal.leakage.ms 46350 &&
mv scienceData.CRACOTest_A.SB46350.CRACOTest_A.beam14_averaged_cal.leakage.ms.tar 46350 &&

mv scienceData.CRACOTest_A.SB46419.CRACOTest_A.beam14_averaged_cal.leakage.ms 46419 &&
mv scienceData.CRACOTest_A.SB46419.CRACOTest_A.beam14_averaged_cal.leakage.ms.tar 46419 &&

mv scienceData.CRACOTest_B.SB46492.CRACOTest_B.beam14_averaged_cal.leakage.ms 46492 &&
mv scienceData.CRACOTest_B.SB46492.CRACOTest_B.beam14_averaged_cal.leakage.ms.tar 46492 &&

mv scienceData.CRACOTest_B.SB46554.CRACOTest_B.beam14_averaged_cal.leakage.ms 46554 &&
mv scienceData.CRACOTest_B.SB46554.CRACOTest_B.beam14_averaged_cal.leakage.ms.tar 46554 &&

mv scienceData.CRACO_ULP2.SB48611.CRACO_ULP2.beam14_averaged_cal.leakage.ms 48511 &&
mv scienceData.CRACO_ULP2.SB48611.CRACO_ULP2.beam14_averaged_cal.leakage.ms.tar 48511

# Initial data lookover
- https://casadocs.readthedocs.io/en/latest/notebooks/data_examination.html

In [ ]:
# open CASA in working directory
casa

# list individual scans including data, date, scans, sources (fields), antenna locations, etc
# IntegrationTime is total duration
# identify an antenna close to the centre with no issues for calibration from the txt file
metadata = listobs(vis=filename, listfile=filename[:-2]+'txt', overwrite=True)


In [ ]:
# @title plot calibrated amplitude vs baseline, verifying data
plotms(vis=filename, xaxis='uvwave', yaxis='amp', ydatacolumn='data', field='0',# avgtime=avgTime/2, correlation='RR',
        overwrite=True)


# Imaging

In [ ]:
# @title image and deconvolute the calibrated target

# tclean() only removes the contributions of the dirty beam
# noise part of the true sky image (residuals) are retained
# effective cleaning occurs at 3-5 image pixels per resolution pixel (12"/2.5"~5)
# target is ~ 9' ~ 540" ~ 216 image pixels
# efficient if the image dimensions are a multiple of 30
# tclean() applies an internal 1.2 multipler on image dimension
# see https://casaguides.nrao.edu/index.php/VLA_Continuum_Tutorial_3C391-CASA5.4.0#Multi-scale_Mosaic_Clean for tutorial on tclean

In [ ]:
ms44857="scienceData.AS113_67.SB44857.AS113_67.beam14_averaged_cal.leakage.ms"
tclean(vis=ms44857,imagename='15OCT2022',
      field='',spw='',
      specmode='mfs',
      niter=20000*10, # or 100, since we have multiple sources within the beam
      gain=0.1, threshold='0.2mJy', # change threshold to 5 times stdev (0.2 mJy, stdev 42 μJy(/beam))
      gridder='standard',
      # deconvolver='multiscale', # remove
      # scales=[0, 6, 18, 54], smallscalebias=0.9, # only relevant for extended sources
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'], # imsize and cell should be such that 1 pixel in the final image corresponds to 1 pixel in the DS9 images
      # imsize is number of pixels, pixel*cellsize = angular size of picture
      # CDELT1 (ra), CDELT2 (dec) in image header are degrees per pixel, correspond to ~2" per pixel
      stokes='IQUV', # keep remaining parameters
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms44918="scienceData.AS113_68.SB44918.AS113_68.beam14_averaged_cal.leakage.ms"
tclean(vis=ms44918,imagename='17OCT2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='IQUV',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms45060="scienceData.AS113_69.SB45060.AS113_69.beam14_averaged_cal.leakage.ms"
tclean(vis=ms45060,imagename='22OCT2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms45086="scienceData.AS113_70.SB45086.AS113_70.beam14_averaged_cal.leakage.ms"
tclean(vis=ms45086, imagename='23OCT2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms45416="scienceData.AS113_71.SB45416.AS113_71.beam14_averaged_cal.leakage.ms"
tclean(vis=ms45416, imagename='05NOV2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms46350="scienceData.CRACOTest_A.SB46350.CRACOTest_A.beam14_averaged_cal.leakage.ms"
tclean(vis=ms46350, imagename='13DEC2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms46419="scienceData.CRACOTest_A.SB46419.CRACOTest_A.beam14_averaged_cal.leakage.ms"
tclean(vis=ms46419, imagename='14DEC2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms46492="scienceData.CRACOTest_B.SB46492.CRACOTest_B.beam14_averaged_cal.leakage.ms"
tclean(vis=ms46492, imagename='15DEC2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms46554="scienceData.CRACOTest_B.SB46554.CRACOTest_B.beam14_averaged_cal.leakage.ms"
tclean(vis=ms46554, imagename='16DEC2022',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
ms48611="scienceData.CRACO_ULP2.SB48611.CRACO_ULP2.beam14_averaged_cal.leakage.ms"
tclean(vis=ms48611, imagename='25FEB2023',
      field='',spw='',
      specmode='mfs',
      niter=20000,
      gain=0.1, threshold='0.2mJy',
      gridder='standard',
      interactive=True,
      imsize=[1800,1800], cell=['2.0arcsec','2.0arcsec'],
      stokes='I',
      weighting='briggs',robust=0.5,
      pbcor=False,
      savemodel='modelcolumn')

In [ ]:
# @title view the target images
# comprised of clean components convolved with a restoring beam and added to the remaining residuals
viewer('15OCT2022'+'.image') # IQUV correction
viewer('17OCT2022'+'.image') # IQUV correction
viewer('22OCT2022'+'.image')
viewer('23OCT2022'+'.image')
viewer('05NOV2022'+'.image')
viewer('13DEC2022'+'.image')
viewer('14DEC2022'+'.image')
viewer('15DEC2022'+'.image')
viewer('16DEC2022'+'.image')
viewer('25FEB2023'+'.image')

# Bugs & Other
-
